In [1]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["SteelArena"]
collection = db["Init"]

In [2]:
# Count the total number of documents
total_documents = collection.count_documents({})
total_documents

2110844

In [3]:
# Get a random document using aggregation with $sample
random_document = collection.aggregate([{"$sample": {"size": 1}}]).next()
# random_document

In [4]:
# Count the number of documents that match the criteria
count = collection.count_documents({"A31.fft": {"$exists": True}})

print(f"Number of documents: {count}")

Number of documents: 65480


In [14]:
pipeline = [
    {"$sort": {"_id": -1}},
    {"$match": {"A31.fft": {"$exists": True}}},
    {"$skip": 0},
    {"$limit": 720}
]

results = list(collection.aggregate(pipeline))

In [15]:
results[0]["time"]

{'datetime': '2024-08-16T10:31:01'}

In [38]:
import plotly.graph_objects as go
from datetime import datetime


def create_spectrum_plot(results, save_name=""):
    """
    Creates a spectrum plot from the given results and optionally saves it as a PNG file.
    
    Parameters:
    - results: List of documents containing the spectrum data.
    - save_name: String representing the filename to save the plot as a PNG. 
                 If empty, the plot will not be saved.
    """
    # Initialize lists to store aggregated frequencies and amplitudes
    all_frequencies = []
    all_amplitudes = []

    # Extract date from the first document's 'datetime' field and reformat it
    date_str = results[0]["time"]['datetime'].split(
        'T')[0]  # Extract the date portion
    # Convert to datetime object
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    # Format the date as DD.MMM.YYYY
    formatted_date = date_obj.strftime('%d. %b. %Y')

    # Loop through all documents in the results list
    for document in results:
        spectrum_data = document["A31"]["fft"]["spectrum"]
        frequencies = [entry['f'] for entry in spectrum_data]
        amplitudes = [entry['a'] for entry in spectrum_data]

        # Append data to the aggregated lists
        all_frequencies.extend(frequencies)
        all_amplitudes.extend(amplitudes)

    # Create a scatter plot using Plotly
    fig = go.Figure(data=[go.Scatter(
        x=all_frequencies,
        y=all_amplitudes,
        mode='markers',
        marker=dict(
            size=10,
            color='rgba(255,255,255,0)',  # Transparent inner circle
            # Outer circle color and width
            line=dict(color='#19C3E6', width=2),
            opacity=0.7  # Transparency of the outer circle
        )
    )])



    w_ = 1920
    h_ = w_ * 9 / 16

    # Update the layout for better readability, set the size, and change Y-axis to log scale
    fig.update_layout(
        title={
            'text': 'Spektrum odozvy - Vrchol oblúka',
            'font': {'size': 52}  # Set the title font size
        },
        xaxis_title={
            'text': 'Frekvencia [Hz]',
            'font': {'size': 48}  # Set the x-axis title font size
        },
        yaxis_title={
            'text': 'Amplitúda',
            'font': {'size': 48}  # Set the y-axis title font size
        },
        width=w_,  # Set the width of the plot
        height=h_,  # Set the height of the plot
        yaxis_type="log",  # Set Y-axis to log scale
        # Logarithmic scale from 1e-5 (10^-5) to 1 (10^0)
        yaxis=dict(
            range=[-1.3979, 0],
            dtick=1,
            tickfont={'size': 32}  # Set the y-ticks font size
        ),
        xaxis=dict(
            range=[0, 100],  # Hardcoded X-axis limits from 0 to 100
            tickfont={'size': 32}  # Set the x-ticks font size
        ),
        template='plotly',  # Use the default light theme

        # Background colors
        # Set the figure background color (around the plot)
        paper_bgcolor='#FFFFFF',
        # Set the plot area background color (where data is plotted)
        plot_bgcolor='#EEEEEE',


        annotations=[
            go.layout.Annotation(
                text=formatted_date,  # Display the formatted date
                xref="paper", yref="paper",
                x=1, y=0,  # Position it at the bottom right corner
                showarrow=False,
                font=dict(size=34, color="black")  # Set the annotation font size
            )
        ]
    )


    # Show the plot
    # fig.show()

    # Save the figure as a PNG file if save_name is not empty
    if save_name:
        fig.write_image(f"{save_name}.png")

# Example usage:
create_spectrum_plot(results, save_name="kokso")  # This will save the plot as "combined_spectrum_plot.png"
# create_spectrum_plot(results)  # This will not save the plot

In [8]:
pipeline = [
    {"$match": {"A31.fft": {"$exists": True}}},
    {"$project": {"date": {"$dateToString": {"format": "%Y-%m-%d",
                                             "date": {"$dateFromString": {"dateString": "$time.datetime"}}}}}},
    {"$group": {"_id": "$date"}},
    {"$count": "unique_days"}
]

result = list(collection.aggregate(pipeline))

# Extract the count of unique days
unique_days_count = result[0]['unique_days'] if result else 0

print(f"Number of unique days: {unique_days_count}")

Number of unique days: 24


In [39]:
for i in range(96*6):
    pipeline = [
        {"$sort": {"_id": -1}},
        {"$match": {"A31.fft": {"$exists": True}}},
        {"$skip": i*120},
        {"$limit": 720}
    ]

    results = list(collection.aggregate(pipeline))

    create_spectrum_plot(results, save_name=str(i))

TypeError: 'NoneType' object is not subscriptable